In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error

# Check stationarity (from https://www.geeksforgeeks.org/sarima-seasonal-autoregressive-integrated-moving-average/)
def check_stationarity(timeseries):
    # Perform the Dickey-Fuller test
    result = adfuller(timeseries, autolag='AIC')
    p_value = result[1]
    print(f'ADF Statistic: {result[0]}')
    print(f'p-value: {p_value}')
    print('Stationary' if p_value < 0.05 else 'Non-Stationary')

def diff(timeseries, num):
    diff_timeseries = timeseries.diff(periods=num)
    diff_timeseries.dropna(inplace=True)
    return diff_timeseries

def diff_n(timeseries, num, n):
    # carry out diff() n times
    for _ in range(n):
        diff_timeseries = diff(timeseries, num)
    return diff_timeseries

def split_data(data: pd.DataFrame, timescale: str):
    timescale_carbon = data.resample(timescale).mean()
    train_data = timescale_carbon.truncate(after=pd.Timestamp('2023-12-31 23:59:59').tz_localize('UTC'))
    test_data = timescale_carbon.truncate(before=pd.Timestamp('2024-01-01 00:00:00').tz_localize('UTC'))
    return train_data, test_data

def forecast_SARIMA(train_data, forecast_periods, params, seasonal_params):
    ## Define SARIMA parameters
    # SARIMA(0,1,1)(0,1,1)12 model
    #p, d, q = 0, 1, 1
    #P, D, Q, s = 0, 1, 1, 12


    # Fit the SARIMA model
    model = SARIMAX(train_data, order=params, seasonal_order=seasonal_params)
    results = model.fit()

    forecast = results.get_forecast(steps=forecast_periods)
    return forecast

def calculate_errors(observed, forecast_mean):
    mae = mean_absolute_error(observed, forecast_mean)
    mse = mean_squared_error(observed, forecast_mean)
    rmse = root_mean_squared_error(observed, forecast_mean)
    return mae, mse, rmse

def plot_forecast(train_data, test_data, forecast):
    # Plot the forecast
    plt.figure(figsize=(20, 7))
    plt.plot(train_data, 'bo-', label='Observed')
    plt.plot(forecast.predicted_mean, 'rx-', label='Forecast')
    plt.plot(test_data, 'ko--', label='Actual')
    plt.fill_between(forecast.conf_int().index, forecast.conf_int().iloc[:, 0], forecast.conf_int().iloc[:, 1], color='pink')
    plt.title("Carbon Intensity Forecast for 2024")
    plt.xlabel("Date")
    plt.ylabel("Carbon Intensity")
    plt.legend()
    plt.show()

if __name__ == '__main__':
    scale = 'D'
    forecast_periods = 348
    m = 365

    ## Get data file path
    file_name = 'df_fuel_ckan.csv'
    file_path = os.path.join(os.getcwd(), 'data', file_name)
    file_path

    ## Read data
    df = pd.read_csv(file_path)
    carbon_data = pd.DataFrame(df['CARBON_INTENSITY'].values, index=pd.to_datetime(df['DATETIME']))

    train_data, test_data = split_data(carbon_data, scale)
    results = []
    forecasts = []
    states = (0, 1)
    for p in states:
        for d in states:
            for q in states:
                for P in states:
                    for D in states:
                        for Q in states:
                            params = (p, d, q)
                            seasonal_params = (P, D, Q, m)
                            forecast = forecast_SARIMA(train_data, forecast_periods, params, seasonal_params)
                            mae, mse, rmse = calculate_errors(test_data, forecast.predicted_mean)
                            results.append((rmse, params, seasonal_params))
                            forecasts.append(forecast)
    #params = (0, 1, 1)
    #seasonal_params = (0, 1, 1, m)
    #print('forecasting...')
    #forecast = forecast_SARIMA(train_data, forecast_periods, params, seasonal_params)
    #mae, mse, rmse = calculate_errors(test_data, forecast.predicted_mean)
    #results.append((rmse, params, seasonal_params))
    #forecasts.append(forecast)
    for result in results:
        print(result[0])
        print()

    print(min(results, key=lambda x: x[0]))
    i = results.index(min(results, key=lambda x: x[0]))
    plot_forecast(train_data, test_data, forecasts[i])
    
    

    #params = (0, 1, 1)
    #seasonal_params = (0, 1, 1, 12)
    #forecast = forecast_SARIMA(train_data, 12)
    #mae, mse, rmse = calculate_errors(test_data, forecast)

    #print(f'MAE: {mae}')
    #print(f'MSE: {mse}')
    #print(f'RMSE: {rmse}')
    #print()

forecasting...


KeyboardInterrupt: 

In [3]:
# Define SARIMA parameters
params = (0, 1, 1)
seasonal_params = (0, 1, 1, 365)

# Forecast daily carbon intensity values for the year 2024
forecast_periods = 365  # Number of days in 2024
forecast = forecast_SARIMA(train_data, forecast_periods, params, seasonal_params)

# Plot the forecast
plot_forecast(train_data, test_data, forecast)

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.82332D+00    |proj g|=  1.50481D-02


KeyboardInterrupt: 